In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import logging

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pvi.models import SparseGaussianProcessRegression
from pvi.clients import Client
from pvi.distributions import MultivariateGaussianDistributionWithZ
from pvi.distributions import MultivariateGaussianFactorWithZ
from pvi.models.kernels import RBFKernel

import torch
import numpy as np
import matplotlib.pyplot as plt
import gpytorch
import tqdm.auto as tqdm

from torch import nn

%matplotlib inline
torch.set_default_dtype(torch.float64)

# Set up data and helper functions

In [ ]:
x = np.linspace(-1, 1, 50)
y = 2 * np.sin(5*x) + 3 * np.abs(x) * np.random.rand(len(x))

x = torch.tensor(x).unsqueeze(1)
y = torch.tensor(y).unsqueeze(1)

In [ ]:
def plot_data(x, y):
    plt.figure()
    plt.grid(b=True)
    plt.scatter(x, y)
    plt.show()
    
def plot_results(x, y, model, q):
    pp = model(x, q)
    mean = pp.mean.detach()
    std = pp.variance.detach() ** 0.5
    
    samples = pp.sample((20,))
    
    plt.figure()
    plt.grid(b=True)
    
    for sample in samples:
        plt.plot(x, sample, color='k', alpha=.1)
        
    plt.plot(x.squeeze(-1), mean)
    plt.fill_between(x.squeeze(-1), mean-1.96*std, mean+1.96*std, alpha=.25)
    plt.scatter(x, y)
    plt.show()
    
def plot_training(training_array):
    x_vals = np.arange(1, len(training_array)+1)
    plt.figure()
    plt.grid(b=True)
    plt.plot(x_vals, training_array)
    plt.ylabel('ELBO Loss')
    plt.xlabel('Step')
    plt.show()
    
data = {
    "x": x,
    "y": y,
}

In [ ]:
plot_data(x, y)

# Construct SGP model

In [ ]:
model_config = {
    "D": 1,
    "kernel_params": {
        "train_hypers": True,
    }
}

model_hyperparameters = {
    "outputsigma": .1,
    "outputscale": 1.,
    "lengthscale": .5,
}

model = SparseGaussianProcessRegression(hyperparameters=model_hyperparameters, config=model_config, train_sigma=True)


inducing_locations = x[::10].clone()
num_inducing = len(inducing_locations)
q = MultivariateGaussianDistributionWithZ(
    nat_params={
        "np1": torch.zeros(num_inducing),
        "np2": -0.5 * torch.eye(num_inducing)
    },
    inducing_locations=inducing_locations,
    train_inducing=False,
)

In [ ]:
model.hyperparameters

In [ ]:
plot_results(x, y, model, q)

# Fit data

In [ ]:
t = MultivariateGaussianFactorWithZ(
    nat_params = {
        "np1": torch.tensor([0.] * num_inducing),
        "np2": torch.tensor([0.] * num_inducing).diag_embed()
    },
    inducing_locations=inducing_locations,
    train_inducing=False,
)

In [ ]:
print(t.nat_params)

In [ ]:
client_config = {
    "epochs": 200,
    "optimiser_params": {"lr": 1e-1},
    "batch_size": 50,
    "train_model": True,
}

# Construct client.
client = Client(data=data, model=model, t=t, config=client_config)

## Without optimising model (hyper-)parameters

In [ ]:
q_new, t_new = client.update_q(q)

In [ ]:
plot_results(x, y, client.model, q_new)